In [46]:
!pip install --quiet google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

from google.colab import auth
from googleapiclient.discovery import build
import re

auth.authenticate_user()
docs_service = build('docs', 'v1')

markdown_text = """
# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues
### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

document = docs_service.documents().create(body={'title': 'Product Team Sync - May 15, 2023'}).execute()
document_id = document.get('documentId')

requests = []
index = 1
checkbox_paragraphs = []
mention_positions = []

def insert_paragraph(text, heading=None):
    global index
    if not text.strip():
        return None, None
    requests.append({
        'insertText': {
            'location': {'index': index},
            'text': text + '\n'
        }
    })
    start = index
    end = index + len(text) + 1
    index = end
    if heading:
        requests.append({
            'updateParagraphStyle': {
                'range': {'startIndex': start, 'endIndex': end},
                'paragraphStyle': {'namedStyleType': heading},
                'fields': 'namedStyleType'
            }
        })
    return start, end

lines = [line.rstrip() for line in markdown_text.strip().split('\n')]
i = 0
while i < len(lines):
    line = lines[i]
    stripped = line.strip()

    if stripped.startswith('# '):
        insert_paragraph(stripped[2:], 'HEADING_1')
    elif stripped.startswith('## '):
        insert_paragraph(stripped[3:], 'HEADING_2')
    elif stripped.startswith('### '):
        insert_paragraph(stripped[4:], 'HEADING_3')
    elif stripped.startswith('- [ ] '):
        text = stripped[6:]
        start, end = insert_paragraph(text)
        if start:
            checkbox_paragraphs.append((start, end))
            for m in re.finditer(r'(@[a-zA-Z]+)', text):
                mention_positions.append((start + m.start(), start + m.end()))
    elif stripped.startswith('- ') or stripped.startswith('* '):
        text = stripped[2:]
        insert_paragraph(text)
    elif stripped == '---':
        requests.append({
            'insertText': {'location': {'index': index}, 'text': '\n'}
        })
        index += 1
    elif stripped.startswith('Meeting recorded by:') or stripped.startswith('Duration:'):
        start, end = insert_paragraph(stripped)
        if start:
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': start, 'endIndex': end - 1},
                    'textStyle': {'italic': True},
                    'fields': 'italic'
                }
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': start, 'endIndex': end},
                    'paragraphStyle': {'alignment': 'END'},
                    'fields': 'alignment'
                }
            })
    else:
        insert_paragraph(stripped)
    i += 1

# Apply checkboxes paragraph by paragraph
for start, end in checkbox_paragraphs:
    requests.append({
        'createParagraphBullets': {
            'range': {'startIndex': start, 'endIndex': end},
            'bulletPreset': 'BULLET_CHECKBOX'
        }
    })

# Style @mentions bold and blue
for start, end in mention_positions:
    requests.append({
        'updateTextStyle': {
            'range': {'startIndex': start, 'endIndex': end},
            'textStyle': {
                'bold': True,
                'foregroundColor': {'color': {'rgbColor': {'red': 0.0, 'green': 0.0, 'blue': 1.0}}}
            },
            'fields': 'bold,foregroundColor'
        }
    })

try:
    docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    print("Google Doc created successfully!")
    print(f"https://docs.google.com/document/d/{document_id}/edit")
except Exception as e:
    print("Error:", e)


Google Doc created successfully!
https://docs.google.com/document/d/1xrgy3dt4J4l67MSfH4Anh9xQ7P0kB-hyEmu_fuJChNM/edit
